In [1]:
from models.llama import *
llama_model, llama_tokenizer = get_model_and_tokenizer("7b")


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
good_tokens_str = list("0123456789, ")
good_tokens = [llama_tokenizer.convert_tokens_to_ids(token) for token in good_tokens_str]
# good_tokens = llama_tokenizer.encode(good_tokens_str)

good_tokens += [llama_tokenizer.eos_token_id]
bad_tokens = [i for i in range(len(llama_tokenizer)) if i not in good_tokens]
print(good_tokens)

[29900, 29896, 29906, 29941, 29946, 29945, 29953, 29955, 29947, 29929, 29892, 0, 2]


In [ ]:
input_str = "Physics is about the study of"
# input_str = "2 5 9 , 2 7 1 , 3 0 0 , 2 9 4 , 2 7 7 , 3 0 7 , 3 3 4"
input_str = "259,271,300,294,277,307,334"
# input_str = "0,1,2,3,4,5,6"


batch = llama_tokenizer([input_str], return_tensors="pt")

batch = {k: v.cuda() for k, v in batch.items()}

generate_ids = llama_model.generate(
            **batch,
            do_sample=True,
            # max_new_tokens=max_tokens,
            max_new_tokens=10,
            temperature=0.9, 
            top_p=0.9, 
            bad_words_ids=[[t] for t in bad_tokens],
            renormalize_logits=True,
        )
gen_strs = llama_tokenizer.batch_decode(
            generate_ids, 
            skip_special_tokens=True, 
            clean_up_tokenization_spaces=False
        )
gen_strs = [x.replace(input_str, '').strip() for x in gen_strs]


In [ ]:
print(gen_strs)

### Explore Serializer

In [ ]:
from data.serialize import *

In [ ]:
settings=SerializerSettings(base=10, prec=3, signed=True, time_sep=',', bit_sep='')
serialize_arr(np.array([112.15379,117.89963,131.78541,128.91249]), settings)

In [4]:
torch.nn.functional.log_softmax

<function torch.nn.functional.log_softmax(input: torch.Tensor, dim: Optional[int] = None, _stacklevel: int = 3, dtype: Optional[int] = None) -> torch.Tensor>